<a href="https://colab.research.google.com/github/ThiagSampaio/NLP/blob/main/Modelo_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> Modelo de regressão _logistica </h1>

Antes tudo, rode as tres linhas abaixo

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/Analise\ sentimento

/content/drive/MyDrive/Analise sentimento


<h3>1 . Importando bibliotecas </h3>

In [ ]:
#importando bibliotecas prontas
import re
import string
import numpy as np
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

from getting_data import *
from Processamento import *
from Processamento_treinamento import *
from nomes_brasileiros import *
from Processamento_expressoes import *
from getting_data import TwitterClient

<h3>2 . Importando dados importantes</h3>

In [ ]:
nltk.download('stopwords')

#importando lista_nomes e presidentes brasileiros. 
lista_presidentes = ["josé sarney", "sarney", "fernando Collor", "collor", "itamar Franco",
                     "fernando henrique cardoso", "luiz inácio lula da silva", "lula", "lulinha",
                     "dilma Rouseff", "dilmão", "dilmãe", "michel temer", "vampiro", "jair bolsonaro",
                     "birolibo", "bozo", "birobiro", "bozonaro", "bolsonaro"]
lista_nome = lista_nomes() #pct nomes brasileiros
lista_nome = lista_nome + lista_presidentes

#importando lista_expressoes
lista_expressoes_positivas, lista_expressoes_negativas = lista_expressoes()

#adicionando termos negativos
termos_negativos = ["genocida", "suicidio", "suicidar", "merda", "bosta", "assassino", "estuprador", "fdp", "racista", "ódio"]
lista_expressoes_negativas = lista_expressoes_negativas + termos_negativos

#pegando dados kaggle sobre tweets positivos e nagativos
lista_positiva,lista_negativa = tratamento()

#Somando todos os termos
lista_positiva = lista_positiva + lista_expressoes_positivas
lista_negativa = lista_negativa + lista_expressoes_negativas

lista_completa  = lista_positiva + lista_negativa

#Criando vetores 
labels = np.append(np.ones((len(lista_positiva))), np.zeros((len(lista_negativa))))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


<h3>3. Funções </h3>

In [ ]:
import string
def process_tweet(tweet, lista_names = lista_nome):
    """Função de processamento de um tweet.
    Input:
        tweet: a string contendo o tweet
    Output:
        tweets_clean: uma lista de palavras contendo o tweet processado

    """
    #pegando lista de palavras stopwords
    stopwords_portuguese = stopwords.words('portuguese')
 
    # remove tickers do mercado de ações como $ GE
    tweet = re.sub(r'\$\w*', '', tweet)
    
    # remove o texto retweetado no estilo antigo "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    
    # remove hashtags
    tweet = re.sub(r'#', '', tweet)
    
    # remove os @
    tweet = re.sub(r'@', '', tweet)

    '''
    # remove os emojis
    padrao_emoji = re.compile("["
                            u"\U0001F600-\U0001F64F"  # emoticons
                            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                            u"\U0001F680-\U0001F6FF"  # transport & map symbols
                            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                            u"\U00002500-\U00002BEF"  # chinese char
                            u"\U00002702-\U000027B0"
                            u"\U00002702-\U000027B0"
                            u"\U000024C2-\U0001F251"
                            u"\U0001f926-\U0001f937"
                            u"\U00010000-\U0010ffff"
                            u"\u2640-\u2642"
                            u"\u2600-\u2B55"
                            u"\u200d"
                            u"\u23cf"
                            u"\u23e9"
                            u"\u231a"
                            u"\ufe0f"  # dingbats
                            u"\u3030"
                            "]+", flags=re.UNICODE)
    tweet = re.sub(padrao_emoji, '', tweet)
   '''
    
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    
    for word in tweet_tokens:
        if (word not in stopwords_portuguese and  # remove stopwords
                word not in string.punctuation and # remove pontuação
                    word not in lista_names):  # remove nomes
            
            tweets_clean.append(word)
    
    return tweets_clean

In [ ]:
# teste da função process_tweet
tweet = "merda 😂 Bolsonaro!!!"
frase = process_tweet(tweet)
print (frase)

['merda', '😂']


In [ ]:
def build_freqs(tweets, ys, lista_names = lista_nome):
    """Crie frequências.
    Entrada:
        tweets: uma lista de tweets
        ys: uma matriz m x 1 com o rótulo de sentimento de cada tweet
            (0 ou 1)
    Saída:
        freqs: um dicionário que mapeia cada par (palavra, sentimento) para seu
        frequência
    """
    # Converta o array np em lista, pois o zip precisa de um iterável.
    # O aperto é necessário ou a lista termina com um elemento.
    # Observe também que este é apenas um NOP se ys já for uma lista.
    yslist = np.squeeze(ys).tolist()

    freqs = {}
    for y, tweet in zip(yslist, tweets):
        for word in process_tweet(tweet,lista_names):
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1

    return freqs

In [ ]:
from math import *

In [ ]:
print(len(lista_positiva))
print(len(lista_negativa))


248276
313952


In [ ]:
'''
import random
for i in range(1,65675):
  lista_negativa.remove(random.choice(lista_negativa))
'''  

KeyboardInterrupt: ignored

In [ ]:
train_pos = lista_positiva
train_neg = lista_negativa

In [ ]:
train_x = train_pos + train_neg 

In [ ]:
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)

In [ ]:
freqs = build_freqs(train_x, train_y)

In [ ]:
import pickle
with open('freqs.json', 'wb') as fp:
    pickle.dump(freqs, fp)

In [ ]:
def sigmoid(z): 
  '''
    Entrada:
        z: é a entrada (pode ser um escalar ou uma matriz)
    Saída:
        h: o sigmóide de z
    '''
    
    h = 1 / (1 + np.exp(-z))
  
    return h

In [ ]:
def gradientDescent(x, y, theta, alpha, num_iters):

    m = x.shape[0]
    
    for i in range(0, num_iters):
       
        z = np.dot(x, theta)
      
        h = sigmoid(z)
        
        J = (-1/m) * (np.dot(y.T, np.log(h)) + np.dot((1 - y).T,np.log(1-h)))

        
        theta = theta - (alpha/m) * (np.dot(x.T,(h - y)))
        
  
    J = float(J)
    return J, theta

In [ ]:
def extract_features(tweet, freqs):

   
    word_l = process_tweet(tweet)
    
   
    x = np.zeros((1, 3)) 
    
    x[0,0] = 1 
    

    for word in word_l:
        

        x[0,1] += freqs.get((word, 1.0), 0)
        
        x[0,2] += freqs.get((word, 0.0),0)
        
    assert(x.shape == (1, 3))
    return x

In [ ]:
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :]= extract_features(train_x[i], freqs)

Y = train_y

# Apply gradient descent
J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 1500)
print(f"The cost after training is {J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(theta)]}")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log


The cost after training is nan.
The resulting vector of weights is [0.0, 0.00012059, -7.062e-05]


In [ ]:
theta

array([[ 4.49412614e-09],
       [ 1.20599068e-04],
       [-7.05937233e-05]])

In [ ]:
import pickle
with open('theta.json', 'wb') as fp:
    pickle.dump(theta, fp)

In [ ]:
def predict_tweet(tweet, freqs, theta):

    x = extract_features(tweet,freqs)
    

    y_pred = sigmoid(np.dot(x,theta))
    
  
    
    return y_pred

In [ ]:
#importando tweets recentes
twitter = []
api = TwitterClient()
tweets = api.get_tweets(query = 'bolsonaro', count = 50)
tweets = dict_to_list(tweets)

lista_tweet_positiva = []
lista_tweet_negativa = []
cont_neg = 0
cont_pos = 0
for i in tweets:

  tweet_tratado = process_tweet(i, lista_names = lista_nome)
  stringu = ' '.join([str(item) for item in tweet_tratado])
  #print(stringu)
  y_hat = predict_tweet(stringu, freqs, theta)
  if y_hat > 0.75:
    #print('Positive sentiment')
    cont_pos += 1
    lista_tweet_positiva.append(i)
  else: 
    #print('Negative sentiment')
    cont_neg += 1
    lista_tweet_negativa.append(i)

In [ ]:
print(len(lista_tweet_negativa))
print(len(lista_tweet_positiva))

43
6


In [ ]:
lista_tweet_positiva

['Missa de sétimo dia do governo Bolsonaro é marcada para sexta que vem https://t.co/iWrBFNzOHc',
 'RT @manugavassi: É óbvio mas sempre bom deixar claro e renovar meus votos (desde sempre) de: FORA BOLSONARO!',
 'RT @De_Pablicia: O DIA N PODERIA COMEÇAR MELHOR \nBOLSONARO CAIU https://t.co/JnSniO1vb8',
 'RT @_anandamiranda: RESUMÃO DA CPI:\n\n1. A Pfizer não foi comprada não por ser cara, mas sim para favorecer a Covaxin\n\n2. A Covaxin não era…',
 'RT @samiabomfim: AGORA! Protesto em São Paulo contra Jair Bolsonaro. A mobilização pelo impeachment tem que ser permanente e não pode esper…',
 'RT @cris_mx31: Bom dia!! O dia já começa com os trend topics perfeitos!! É hoje!!\nBOLSONARO CAIU\nACABOU BOLSONARO https://t.co/dYn9R4Kli9']